In [8]:


import pandas as pd
import pandas as pd
import csv


from google.colab import drive
drive.mount('/content/gdrive')
#with open("/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica 3.xslx", 'rb') as f:
#  df = f.read()

df= pd.read_excel(open("/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica 3.xlsx", 'rb')) 
df.head(10)

#from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf = TfidfVectorizer()
#tfidf.fit(df[])



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Row,NewsID,Title,Text,Date,Time,Category two,Category
0,1,843656,وزير علوم درجمع استادان نمونه: سن بازنشستگي اس...,وزير علوم در جمع استادان نمونه كشور گفت: از اس...,138/5//09,0:9::18,آموزشي-,آموزشي
1,2,837144,گردهمايي دانش‌آموختگان موسسه آموزش عالي سوره ب...,به گزارش سرويس صنفي آموزشي خبرگزاري دانشجويان ...,138/5//09,1:4::11,آموزشي-,آموزشي
2,3,436862,نتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور ا...,نتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و ك...,138/3//07,1:0::03,آموزشي-,آموزشي
3,4,174187,وضعيت اقتصادي و ميزان تحصيلات والدين از مهمتري...,محمدتقي علوي يزدي، مجري اين طرح پژوهشي در اين‌...,138/1//08,1:1::49,آموزشي-,آموزشي
4,5,768531,دانشكده علوم حديث همزمان با سال پيامبر اعظم (ص...,دوره آموزشي سيره نبوي بعنوان يكي از رشته‌هاي ج...,138/5//05,1:1::18,آموزشي-,آموزشي
5,6,395642,نخستين مركز آموزش عالي بين‌المللي دانشگاه مونا...,به گزارش خبرنگار اعزامي خبرگزاري دانشجويان اير...,138/3//03,1:1::52,آموزشي-,آموزشي
6,7,176676,دومين گردهمايي انجمن‌هاي علمي دانشجويي دانشگاه...,به گزارش سرويس آموزشي ايسنا، دفتر انجمن‌هاي عل...,138/1//09,0:9::44,آموزشي-,آموزشي
7,8,787861,تقويم تحصيلي دانشگاه پيام نور مشهد اعلام شد,به گزارش خبرگزاري دانشجويان ايران (ايسنا) - من...,138/5//06,0:8::38,آموزشي-,آموزشي
8,9,142104,هفتمين اجلاس مشترك هياتهاي امناي دانشگاهها و م...,به گزارش خبرنگار آموزشي ايسنا، در اين اجلاس يك...,138/1//04,1:1::58,آموزشي-,آموزشي
9,10,154915,وزارت علوم: خبر فروش رشته‌هاي دانشگاهي روزنامه...,به گزارش سرويس آموزشي ايسنا، در اين نمابر آمده...,138/1//06,1:2::08,آموزشي-,آموزشي


#NEW

In [2]:
import pandas as pd
import csv
import sys
from google.colab import drive


drive.mount('/content/gdrive')


#with open("/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica.csv", 'rb') as f:
#  data1 = f.read()

#corpus =pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica.csv", error_bad_lines=False)



#corona_df = pd.read_csv('data.csv')
#corona_df.dropna(subset=['text_body'],inplace=True)
#corona_df.head()

Mounted at /content/gdrive


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit(df['Text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [10]:
X = tfidf.transform(df['Text'])
df['Text'][1]

'به گزارش سرويس صنفي آموزشي خبرگزاري دانشجويان ايران (ايسنا)، در اين برنامه كه به منظور تاسيس و انتخابات جامعه دانش\u200cآموختگان سوره برگزار مي\u200cشود جمعي از مسئولان و مديران سازمان تبليغات اسلامي، مديران حوزه هنري، مديران موسسه آموزش عالي سوره و همچنين جمعي از هنرمندان، استادان و اعضاي هيات علمي اين موسسه حضور دارند. همچنين در اين برنامه از جمعي از دانش آموختگان نمونه و استادان پيشكسوت اين موسسه تقدير مي\u200cشود. اجراي برنامه\u200cهاي موسيقي، تئاتر و نمايشگاهي از فعاليت\u200cهاي دانشجويان در زمينه\u200cهاي فرهنگي و هنري در گذشته و حال و همچنين غرفه\u200cهاي عكس يادگاري و ذكر خاطره\u200cاي از درگذشتگان سوره از جمله برنامه\u200cهاي جنبي اين گردهمايي مي\u200cباشد.'

In [3]:
"""
======================================================
Create raw text dataset from Reuters
======================================================
This script uses the code from the scikit-learn example 
plot_out_of_core_classification.py for retrieving the Reuters dataset.
The dataset used in this example is Reuters-21578 as provided by the UCI ML
repository. It will be automatically downloaded and uncompressed on first run.
"""

# Authors: Eustache Diemert <eustache@diemert.fr>

# License: BSD 3 clause

from __future__ import print_function

from glob import glob
import itertools
import os.path
import re
import tarfile

import numpy as np

from sklearn.externals.six.moves import html_parser
from sklearn.externals.six.moves import urllib
from sklearn.datasets import get_data_home

import pickle


###############################################################################
# Reuters Dataset related routines
###############################################################################

def _not_in_sphinx():
    # Hack to detect whether we are running by the sphinx builder
    return '__file__' in globals()

class ReutersParser(html_parser.HTMLParser):
    """Utility class to parse a SGML file and yield documents one at a time."""

    def __init__(self, encoding='latin-1'):
        html_parser.HTMLParser.__init__(self)
        self._reset()
        self.encoding = encoding

    def handle_starttag(self, tag, attrs):
        method = 'start_' + tag
        getattr(self, method, lambda x: None)(attrs)

    def handle_endtag(self, tag):
        method = 'end_' + tag
        getattr(self, method, lambda: None)()

    def _reset(self):
        self.in_title = 0
        self.in_body = 0
        self.in_topics = 0
        self.in_topic_d = 0
        self.title = ""
        self.body = ""
        self.topics = []
        self.topic_d = ""

    def parse(self, fd):
        self.docs = []
        for chunk in fd:
            self.feed(chunk.decode(self.encoding))
            for doc in self.docs:
                yield doc
            self.docs = []
        self.close()

    def handle_data(self, data):
        if self.in_body:
            self.body += data
        elif self.in_title:
            self.title += data
        elif self.in_topic_d:
            self.topic_d += data

    def start_reuters(self, attributes):
        pass

    def end_reuters(self):
        self.body = re.sub(r'\s+', r' ', self.body)
        self.docs.append({'title': self.title,
                          'body': self.body,
                          'topics': self.topics})
        self._reset()

    def start_title(self, attributes):
        self.in_title = 1

    def end_title(self):
        self.in_title = 0

    def start_body(self, attributes):
        self.in_body = 1

    def end_body(self):
        self.in_body = 0

    def start_topics(self, attributes):
        self.in_topics = 1

    def end_topics(self):
        self.in_topics = 0

    def start_d(self, attributes):
        self.in_topic_d = 1

    def end_d(self):
        self.in_topic_d = 0
        self.topics.append(self.topic_d)
        self.topic_d = ""


def stream_reuters_documents(data_path=None):
    """Iterate over documents of the Reuters dataset.
    The Reuters archive will automatically be downloaded and uncompressed if
    the `data_path` directory does not exist.
    Documents are represented as dictionaries with 'body' (str),
    'title' (str), 'topics' (list(str)) keys.
    """

    DOWNLOAD_URL = ("/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica.csv")
    ARCHIVE_FILENAME = "/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica.csv"

    if data_path is None:
        data_path = os.path.join(get_data_home(), "reuters")
    if not os.path.exists(data_path):
        """Download the dataset."""
        print("downloading dataset (once and for all) into %s" %
              data_path)
        os.mkdir(data_path)

        def progress(blocknum, bs, size):
            total_sz_mb = '%.2f MB' % (size / 1e6)
            current_sz_mb = '%.2f MB' % ((blocknum * bs) / 1e6)
            if _not_in_sphinx():
                print('\rdownloaded %s / %s' % (current_sz_mb, total_sz_mb),
                      end='')

        archive_path = os.path.join(data_path, data)
        urllib.request.urlretrieve(DOWNLOAD_URL, filename=archive_path,
                                   reporthook=progress)
        if _not_in_sphinx():
            print('\r', end='')
        print("untarring Reuters dataset...")
        tarfile.open(archive_path, 'r:gz').extractall(data_path)
        print("done.")

    parser = ReutersParser()
    for filename in glob(os.path.join(data_path, "*.sgm")):
        for doc in parser.parse(open(filename, 'rb')):
            yield doc

def get_minibatch(doc_iter, size, pos_class):
    """Extract a minibatch of examples, return a tuple X_text, y.
    Note: size is before excluding invalid docs with no topics assigned.
    """
    data = [(u'{title}\n\n{body}'.format(**doc), doc['topics'])
            for doc in itertools.islice(doc_iter, size)
            if doc['topics']]

    # If there's no data, just return empty lists.    
    if not len(data):
        return np.asarray([], dtype=int), np.asarray([], dtype=int).tolist()
    
    # Otherwise, retrieve the articles and class labels. zip just splits apart
    # the two variables.
    X_text, y = zip(*data)

    # Convert X_text and y from tuples to lists.    
    X_text = list(X_text)    
    y = list(y)
    
    # Convert the class labels to a list.
    #y = np.asarray(y, dtype=int).tolist()    
    
    # For some reason, some of these articles are just whitespace. Look for 
    # these and remove them. 
    toRemove = []
    docNum = 0
    
    # For each article...
    for article in X_text:
        # If the article is just whitespace, or is empty, we'll remove it        
        if article.isspace() or (article == ""):
            toRemove.append(docNum)
            
        docNum += 1
    
    # Remove the empty articles. Do this in reverse order so as not to corrupt
    # the indeces as we go.
    toRemove.reverse()
    for i in toRemove:
        del X_text[i]
        del y[i]
    
    return X_text, y



def iter_minibatches(doc_iter, minibatch_size):
    """Generator of minibatches."""
    X_text, y = get_minibatch(doc_iter, minibatch_size)
    while len(X_text):
        yield X_text, y
        X_text, y = get_minibatch(doc_iter, minibatch_size)

###############################################################################
# Main
###############################################################################

# Iterator over parsed Reuters SGML files.
data_stream = stream_reuters_documents()

# The Reuter's dataset includes many different classes, but we're just going to
# do binary classification. We'll use 'acq' (articles related to 
# "acquisitions"--one of the most prevalent classes in the dataset) as the 
# positive class, and all other article topics will be used as negative 
# examples.
positive_class = 'acq'

# Retrieve a set of examples from the dataset to use as the training set, then 
# another set of examples to use as the test set. The actual number will
# be smaller because it will exclude "invalid docs with no topics assigned".
X_train_raw, y_train_raw = get_minibatch(data_stream, 8000, positive_class)
X_test_raw, y_test_raw = get_minibatch(data_stream, 2000, positive_class)

print("Train set is %d documents" % (len(y_train_raw)))
print("Test set is %d documents" % (len(y_test_raw)))

# Dump the dataset to a pickle file.
pickle.dump((X_train_raw, y_train_raw, X_test_raw, y_test_raw), open("/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica.csv", "wb"))



/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


downloading dataset (once and for all) into /root/scikit_learn_data/reuters


NameError: ignored

In [ ]:
#!/usr/bin/env python
"""
Run k-NN classification on the Reuters text dataset using LSA.
This script leverages modules in scikit-learn for performing tf-idf and SVD.
Classification is performed using k-NN with k=5 (majority wins).
The script measures the accuracy of plain tf-idf as a baseline, then LSA to
show the improvement.
@author: Chris McCormick
"""

import pickle
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier


###############################################################################
#  Load the raw text dataset.
###############################################################################

print("Loading dataset...")

# The raw text dataset is stored as tuple in the form:
# (X_train_raw, y_train_raw, X_test_raw, y_test)
# The 'filtered' dataset excludes any articles that we failed to retrieve
# fingerprints for.
raw_text_dataset = pickle.load( open( "/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica.csv", "rb" ) )
X_train_raw = raw_text_dataset[0]
y_train_labels = raw_text_dataset[1] 
X_test_raw = raw_text_dataset[2]
y_test_labels = raw_text_dataset[3]

# The Reuters dataset consists of ~100 categories. However, we are going to
# simplify this to a binary classification problem. The 'positive class' will
# be the articles related to "acquisitions" (or "acq" in the dataset). All
# other articles will be negative.
y_train = ["acq" in y for y in y_train_labels]
y_test = ["acq" in y for y in y_test_labels]

print("  %d training examples (%d positive)" % (len(y_train), sum(y_train)))
print("  %d test examples (%d positive)" % (len(y_test), sum(y_test)))


###############################################################################
#  Use LSA to vectorize the articles.
###############################################################################

# Tfidf vectorizer:
#   - Strips out “stop words”
#   - Filters out terms that occur in more than half of the docs (max_df=0.5)
#   - Filters out terms that occur in only one document (min_df=2).
#   - Selects the 10,000 most frequently occuring words in the corpus.
#   - Normalizes the vector (L2 norm of 1.0) to normalize the effect of 
#     document length on the tf-idf values. 
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                             min_df=2, stop_words='english',
                             use_idf=True)

# Build the tfidf vectorizer from the training data ("fit"), and apply it 
# ("transform").
X_train_tfidf = vectorizer.fit_transform(X_train_raw)

print("  Actual number of tfidf features: %d" % X_train_tfidf.get_shape()[1])

print("\nPerforming dimensionality reduction using LSA")
t0 = time.time()

# Project the tfidf vectors onto the first N principal components.
# Though this is significantly fewer features than the original tfidf vector,
# they are stronger features, and the accuracy is higher.
svd = TruncatedSVD(100)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

print("  done in %.3fsec" % (time.time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))


# Now apply the transformations to the test data as well.
X_test_tfidf = vectorizer.transform(X_test_raw)
X_test_lsa = lsa.transform(X_test_tfidf)


###############################################################################
#  Run classification of the test articles
###############################################################################

print("\nClassifying tfidf vectors...")

# Time this step.
t0 = time.time()

# Build a k-NN classifier. Use k = 5 (majority wins), the cosine distance, 
# and brute-force calculation of distances.
knn_tfidf = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='cosine')
knn_tfidf.fit(X_train_tfidf, y_train)

# Classify the test vectors.
p = knn_tfidf.predict(X_test_tfidf)

# Measure accuracy
numRight = 0;
for i in range(0,len(p)):
    if p[i] == y_test[i]:
        numRight += 1

print("  (%d / %d) correct - %.2f%%" % (numRight, len(y_test), float(numRight) / float(len(y_test)) * 100.0))

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)
print("  done in %.3fsec" % elapsed)


print("\nClassifying LSA vectors...")

# Time this step.
t0 = time.time()

# Build a k-NN classifier. Use k = 5 (majority wins), the cosine distance, 
# and brute-force calculation of distances.
knn_lsa = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='cosine')
knn_lsa.fit(X_train_lsa, y_train)

# Classify the test vectors.
p = knn_lsa.predict(X_test_lsa)

# Measure accuracy
numRight = 0;
for i in range(0,len(p)):
    if p[i] == y_test[i]:
        numRight += 1

print("  (%d / %d) correct - %.2f%%" % (numRight, len(y_test), float(numRight) / float(len(y_test)) * 100.0))

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)    
print("    done in %.3fsec" % elapsed)

In [ ]:
#!/usr/bin/env python
"""
Perform some analysis on the top components of SVD.
This script takes articles from the Reuters classification dataset, then
applies LSA to them to create compact feature vectors.
We look at some properties of these vectors and the SVD matrix in order to gain
some insight into how they work.
@author: Chris McCormick
"""

import pickle
import time
import numpy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

from pylab import *

import random

###############################################################################
#  Load the raw text dataset.
###############################################################################

print("Loading dataset...")

# The raw text dataset is stored as tuple in the form:
# (X_train_raw, y_train_raw, X_test_raw, y_test)
# The 'filtered' dataset excludes any articles that we failed to retrieve
# fingerprints for.
raw_text_dataset = pickle.load( open( "/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica.csv", "rb" ) )
X_train_raw = raw_text_dataset[0]

print("  %d training examples" % (len(X_train_raw)))

###############################################################################
#  Use LSA to vectorize the articles.
###############################################################################

# Tfidf vectorizer:
#   - Strips out “stop words”
#   - Filters out terms that occur in more than half of the docs (max_df=0.5)
#   - Filters out terms that occur in only one document (min_df=2).
#   - Selects the 10,000 most frequently occuring words in the corpus.
#   - Normalizes the vector (L2 norm of 1.0) to normalize the effect of 
#     document length on the tf-idf values. 
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                             min_df=2, stop_words='english',
                             use_idf=True)

# Build the tfidf vectorizer from the training data ("fit"), and apply it 
# ("transform").
X_train_tfidf = vectorizer.fit_transform(X_train_raw)

print("  Actual number of tfidf features: %d" % X_train_tfidf.get_shape()[1])

# Get the words that correspond to each of the features.
feat_names = vectorizer.get_feature_names()

# Print ten random terms from the vocabulary
print("Some random words in the vocabulary:")
for i in range(0, 10):
    featNum = random.randint(0, len(feat_names))
    print("  %s" % feat_names[featNum])
    
print("\nPerforming dimensionality reduction using LSA")
t0 = time.time()

# Project the tfidf vectors onto the first N principal components.
# Though this is significantly fewer features than the original tfidf vector,
# they are stronger features, and the accuracy is higher.
svd = TruncatedSVD(100)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

# The SVD matrix will have one row per component, and one column per feature
# of the original data.

#for compNum in range(0, 100, 10):
for compNum in range(0, 10):

    comp = svd.components_[compNum]
    
    # Sort the weights in the first component, and get the indeces
    indeces = numpy.argsort(comp).tolist()
    
    # Reverse the indeces, so we have the largest weights first.
    indeces.reverse()
    
    # Grab the top 10 terms which have the highest weight in this component.        
    terms = [feat_names[weightIndex] for weightIndex in indeces[0:10]]    
    weights = [comp[weightIndex] for weightIndex in indeces[0:10]]    
   
    # Display these terms and their weights as a horizontal bar graph.    
    # The horizontal bar graph displays the first item on the bottom; reverse
    # the order of the terms so the biggest one is on top.
    terms.reverse()
    weights.reverse()
    positions = arange(10) + .5    # the bar centers on the y axis
    
    figure(compNum)
    barh(positions, weights, align='center')
    yticks(positions, terms)
    xlabel('Weight')
    title('Strongest terms for component %d' % (compNum))
    grid(True)
    show()

In [ ]:
lines.append(line[:-1])
class:
			lines.append(line)
				yield {
						'id': int(lines[0]),
						'title': lines[1],
						'text': lines[2],
						'date': lines[3],
						'time': lines[4],
						'category': lines[5],
						'category2': lines[6],
					}
					lines = []

	def texts(self):
		for doc in self.docs():
			yield doc['text']
    """
    interfaces [Persica Corpus](https://sourceforge.net/projects/persica/)
    >>> persica = PersicaReader('corpora/persica.csv')
    >>> next(persica.docs())['id']
    843656
    """

    def __init__(self, csv_file):
        self._csv_file = csv_file

    def docs(self):
        lines = []
        for line in codecs.open(self._csv_file, encoding='utf-8-sig'):
            line = line.strip()
            if line:
                if line.endswith(','):
                    lines.append(line[:-1])
                else:
                    lines.append(line)
                    yield {
                        'id': int(lines[0]),
                        'title': lines[1],
                        'text': lines[2],
                        'date': lines[3],
                        'time': lines[4],
                        'category': lines[5],
                        'category2': lines[6],
                    }
                    lines = []

    def texts(self):
        for doc in self.docs():
            yield doc['text']

In [ ]:
import pandas as pd
#!pip install hazm
import hazm
import matplotlib.pyplot as plt
#from stopwords import all_stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report


normalizer = hazm.Normalizer(persian_numbers=True)
tokenizer = hazm.WordTokenizer(replace_numbers=True, replace_hashtags=True)
lemmatizer = hazm.Lemmatizer()
tagger = hazm.POSTagger(model= "/content/gdrive/My Drive/Colab Notebooks/NLP/HW03/persica.csv")

def clean_text(sentence):
    sentence = normalizer.normalize(sentence)
    sentence = tokenizer.tokenize(sentence)
    sentence = tagger.tag(sentence)
    sentence = [lemmatizer.lemmatize(x[0], pos=x[1]) for x in sentence]
    return " ".join(sentence)

data = pd.read_csv('Corpus/persica_org_english_cat.csv')
data = data[['title', 'text', 'category2']].dropna()
data['cleaned_text'] = data["title"] + " " + data['text']

doc = ""
for item in data['cleaned_text']:
    doc += item + " "
doc = doc.split()

print("Number of Entries: {:,}\nNumber of tokens: {:,}\nVocabulary size: {:,}\nNumber of Classes: {:,}\n".format(
    data.shape[0], 
    len(doc),
    len(set(doc)),
    len(set(data['category2']))
))

le = LabelEncoder()
data['num_cat'] = le.fit_transform(data['category2'])

# Shuffling the Data to get a more realistic evaluation
data = data.sample(frac=1)

# Defining Features and Labels
X = data['cleaned_text']
y = data['num_cat']

# Spliting the Data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8)

# The PipeLine Containing the Vectorizer and The Model With Text PreProcessing
sgd = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(2, 3), lowercase=False, max_df=0.1, preprocessor=clean_text,
                              stop_words=all_stopwords)),
    ('clf', SGDClassifier(loss='modified_huber', max_iter=1300, n_jobs=-1, n_iter_no_change=80)),
])

# Fitting the Model with Training Data
sgd.fit(X_train, y_train)

# Making Predictions Using the Model on Test Data
pred = sgd.predict(X_test)

print(classification_report(y_test, pred, target_names=list(le.classes_)))